In [ ]:
##Import this library
!git clone https://github.com/astutic/brat-standoff-to-json.git

In [ ]:
### Copy the annotation.conf into your desired dataset folder 
# Change the annotation.conf
!cp /content/brat-standoff-to-json/testData/CoNLL-ST_2002/annotation.conf "/path/to/ANN_and_TXT/"

In [ ]:
#To check which content the annotation.conf have
!cat /your/path/to/annotation.conf

#Go Installation Above repo works on GO language.

In [ ]:
# run this cell first time using python runtime
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!npx degit gopherdata/gophernotes/kernel \
     /usr/local/share/jupyter/kernels/gophernotes
# then refresh, it will now use gophernotes. Skip to golang in later cells

###Go to your folder and make a json dataset.

In [ ]:
%cd /content/brat-standoff-to-json/
!pwd
!go run main.go -p "/your/folder/to/annotation/folder/" -f --output "/yourfolder/filename.json"
#example below
#!go run main.go -p "/content/total/train/" -f --output "/content/train.json"

###Now you have made .json file 

In [ ]:
import json
mk = open("test.json","r")
df = mk.readlines()
#df.close()

main_list=[]

for i in df:
  a=json.loads(i)
  b=a["Entities"]
  ent={"entities":[tuple(x) for x in b]}
  ll = list()
  ll.append(a["Data"])
  ll.append(ent)
  final = tuple(ll)
  main_list.append(final)

###!^ Above will give you file like this 
('I am Surjit', {(6,11,"PER")}

You can change above tuple to normal python list.
**bold text**

###check some example below

In [ ]:
for i in main_list[0:1]:
  print(i)

#Its time for making train.spacy files

In [ ]:
import spacy
!python3 -m spacy download en_core_web_lg #any version you want

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_lg") # load other spacy model

db = DocBin() # create a DocBin object
count=0
for text, annot in tqdm(returned_data): # data in previous format
    count = count+1
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label)#, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    print(ents)
    print(count)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir('/content/')
db.to_disk("./surjittest.spacy") # save the docbin object

#Value Error : Span not matched or leading and trailing error etc.
Run Below Cell code first.


#Span not matched or same entiy span used two time.
just pop that item from list eq:
main_list.pop(index_number)
OR correct manually


In [ ]:
import re
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [ ]:
ouput_data = trim_entity_span(main_list)